In [1]:
from queue import Queue
import time

In [2]:
def read_input(filename):
    with open(filename, 'r') as f:
        d = [s.rstrip() for s in f.readlines()]
        d = [[s for s in ss] for ss in d]
    ncol = len(d[0])
    nrow = len(d)
    r0 = nrow // 2
    c0 = ncol // 2
    tunnel_map = {(i, j): d[i][j] for i in range(nrow) for j in range(ncol)}
    d1 = [r[:ncol // 2 + 1] for r in d[:nrow // 2 + 1]]
    d2 = [r[ncol // 2:] for r in d[:nrow // 2 + 1]]
    d3 = [r[:ncol // 2 + 1] for r in d[nrow // 2:]]
    d4 = [r[ncol //2:] for r in d[nrow //2:]]
    k1 = [k for k in set([l for s in d1 for l in s]) - set(['#', '.', '@']) if k.islower()]
    k2 = [k for k in set([l for s in d2 for l in s]) - set(['#', '.', '@']) if k.islower()]
    k3 = [k for k in set([l for s in d3 for l in s]) - set(['#', '.', '@']) if k.islower()]
    k4 = [k for k in set([l for s in d4 for l in s]) - set(['#', '.', '@']) if k.islower()]
    pstart1 = ((r0 - 1, c0 - 1), frozenset(k2 + k3 + k4))
    pstart2 = ((r0 - 1, c0 + 1), frozenset(k1 + k3 + k4))
    pstart3 = ((r0 + 1, c0 - 1), frozenset(k1 + k2 + k4))
    pstart4 = ((r0 + 1, c0 + 1), frozenset(k1 + k2 + k3))
    return (pstart1, pstart2, pstart3, pstart4), tunnel_map

In [3]:
def get_adjacent(p, tunnel_map, all_keys):
    (x, y), keys = p
    doors = [k.upper() for k in keys]
    adj = []
    for p_a in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
        if tunnel_map[p_a] in ['.', '@'] + doors + all_keys:
            adj.append((p_a, keys))
    return adj

def bfs(tunnel_map, p_start, all_keys):
    q = Queue()
    q.put(p_start)
    explored = set()
    dist = {p_start: 0}
    while not q.empty():
        v = q.get()
        xy, keys = v
        if tunnel_map[xy] in all_keys:
            if not tunnel_map[xy] in keys:
                v_old = v
                v = (xy, keys | frozenset(tunnel_map[xy]))
                dist[v] = dist[v_old]
        if set(v[1]) == set(all_keys):
            return dist[v]
        for w in get_adjacent(v, tunnel_map, all_keys):
            if not w in explored:
                explored.add(w)
                dist[w] = dist[v] + 1
                q.put(w)
    return -1

In [4]:
def runit(filename):
    p_starts, tunnel_map = read_input(filename)
    all_keys = [c for c in list(set(tunnel_map.values()) - set(['#', '.', '@'])) if c.islower()]
    t0 = time.time()
    dist = 0
    for p_start in p_starts:
        dist += bfs(tunnel_map, p_start, all_keys)
    dt = time.time() - t0
    res = {'time': dt, 'distance': dist}
    return res

In [7]:
res = runit('18_input2.txt')
res

{'time': 0.22335100173950195, 'distance': 2348}